# 🧩 Aula 1 – Pensando em Paralelismo e Performance (CPU)

## Computação de Alto Desempenho em Python para Engenharia Civil

**Objetivos desta aula:**
- Entender o que é paralelismo e onde ele aparece em problemas de engenharia
- Introduzir conceitos de speedup, escalabilidade e overhead
- Escrever primeiros exemplos em Python puro e com paralelismo de CPU
- Aplicar multiprocessing em problemas reais

---

### 🎯 Por que Engenheiros Civis devem se importar com HPC?

- **Simulações estruturais** (FEM - Finite Element Method)
- **Análise de fluxo** (CFD - Computational Fluid Dynamics) 
- **Transporte de calor** e difusão
- **Simulações Monte Carlo** para análise de confiabilidade
- **Crescimento exponencial** do tamanho dos problemas

**Pergunta motivadora:** *O que acontece se eu dobrar o número de nós na minha malha?*

In [ ]:
# Import Required Libraries
import time
import numpy as np
import matplotlib.pyplot as plt
import threading
import multiprocessing as mp
from concurrent.futures import ProcessPoolExecutor, as_completed
import psutil
import os

# Configure matplotlib for better plots
plt.style.use('seaborn-v0_8' if 'seaborn-v0_8' in plt.style.available else 'default')
plt.rcParams['figure.figsize'] = (10, 6)

print(f"Python está executando com {mp.cpu_count()} núcleos de CPU disponíveis")
print(f"Versão NumPy: {np.__version__}")
print(f"Sistema: {psutil.cpu_count()} CPUs, {psutil.virtual_memory().total // (1024**3)} GB RAM")

## 1. Da Computação Serial ao Paralelismo

### 🔄 Conceitos Fundamentais

**Computação Serial:**
- CPU = 1 núcleo → 1 tarefa por vez
- Execução sequencial, passo a passo

**Computação Paralela:**
- Computadores modernos = múltiplos núcleos → tarefas simultâneas
- Divisão do trabalho entre processos/threads

### 📊 Métricas de Performance

- **Speedup (S)**: $S = \frac{T_{serial}}{T_{paralelo}}$
- **Eficiência (E)**: $E = \frac{S}{P} = \frac{T_{serial}}{P \times T_{paralelo}}$
- **Overhead**: Tempo extra gasto na coordenação entre processos

### ⚖️ Leis Fundamentais

**Lei de Amdahl**: $S_{max} = \frac{1}{f + \frac{(1-f)}{P}}$
- onde `f` = fração serial, `P` = número de processadores

**Lei de Gustafson**: $S = f + P(1-f)$
- Considera que o problema cresce com os recursos disponíveis

## 2. Exemplo 1: Soma de Vetores - Serial vs NumPy vs Threading

Vamos começar com um exemplo clássico: somar dois vetores grandes elemento por elemento.

In [ ]:
def vector_sum_serial(a, b):
    """Soma serial elemento por elemento"""
    start = time.perf_counter()
    c = np.zeros_like(a)
    for i in range(len(a)):
        c[i] = a[i] + b[i]
    end = time.perf_counter()
    return c, end - start

def vector_sum_numpy(a, b):
    """Soma usando NumPy (paralelismo implícito)"""
    start = time.perf_counter()
    c = a + b
    end = time.perf_counter()
    return c, end - start

def vector_sum_chunk_worker(args):
    """Worker para soma paralela por chunks"""
    a_chunk, b_chunk, start_idx = args
    result = a_chunk + b_chunk
    return start_idx, result

def vector_sum_threading(a, b, num_threads=4):
    """Soma usando threading (limitado pelo GIL)"""
    start = time.perf_counter()
    
    chunk_size = len(a) // num_threads
    threads = []
    results = [None] * num_threads
    
    def worker(thread_id):
        start_idx = thread_id * chunk_size
        end_idx = start_idx + chunk_size if thread_id < num_threads - 1 else len(a)
        
        for i in range(start_idx, end_idx):
            results[thread_id] = (a[start_idx:end_idx] + b[start_idx:end_idx])
    
    # Criar e iniciar threads
    for i in range(num_threads):
        thread = threading.Thread(target=worker, args=(i,))
        threads.append(thread)
        thread.start()
    
    # Aguardar conclusão
    for thread in threads:
        thread.join()
    
    # Combinar resultados
    c = np.concatenate([r for r in results if r is not None])
    end = time.perf_counter()
    return c, end - start

# Teste com vetores grandes
N = 10_000_000
print(f"Criando vetores de tamanho {N:,}...")
a = np.arange(N, dtype=np.float64)
b = np.arange(N, dtype=np.float64)

print("\\nComparando métodos de soma de vetores:")
print("=" * 50)

# Método serial (apenas para vetores menores)
if N <= 1_000_000:
    _, time_serial = vector_sum_serial(a[:1_000_000], b[:1_000_000])
    print(f"Serial (1M elementos):     {time_serial:.3f}s")

# Método NumPy
_, time_numpy = vector_sum_numpy(a, b)
print(f"NumPy ({N//1_000_000}M elementos):      {time_numpy:.3f}s")

# Método Threading (mostra limitação do GIL)
_, time_threading = vector_sum_threading(a, b, num_threads=4)
print(f"Threading ({N//1_000_000}M elementos):   {time_threading:.3f}s")

print(f"\\nSpeedup NumPy vs Threading: {time_threading/time_numpy:.2f}x")
print("⚠️  Note que threading não oferece speedup para operações CPU-bound em Python!")

## 3. Arquitetura de CPU e o Global Interpreter Lock (GIL)

### 🏗️ Hierarquia de Memória
- **Registradores** (mais rápido, menor capacidade)
- **Cache L1/L2/L3** (rápido, capacidade média)
- **RAM** (médio, grande capacidade)  
- **Storage** (mais lento, maior capacidade)

### 🧵 Threads vs Processos

**Threads:**
- Compartilham o mesmo espaço de memória
- Comunicação rápida, mas sincronização complexa
- **Limitação em Python**: GIL (Global Interpreter Lock)

**Processos:**
- Espaços de memória separados
- Comunicação via IPC (Inter-Process Communication)
- Cada processo tem seu próprio interpretador Python

### ⚠️ O Problema do GIL
O GIL permite que apenas uma thread execute código Python por vez, limitando o paralelismo real para tarefas CPU-bound.

## 4. Exemplo 2: Estimativa de π usando Monte Carlo com Multiprocessing

O método Monte Carlo é ideal para demonstrar paralelismo real, pois cada processo pode trabalhar independentemente.

In [ ]:
def monte_carlo_pi_serial(n_samples):
    """Estimativa de π serial usando Monte Carlo"""
    start = time.perf_counter()
    
    np.random.seed(42)  # Para reprodutibilidade
    x = np.random.uniform(-1, 1, n_samples)
    y = np.random.uniform(-1, 1, n_samples)
    
    # Pontos dentro do círculo unitário
    inside_circle = (x**2 + y**2) <= 1
    pi_estimate = 4 * np.sum(inside_circle) / n_samples
    
    end = time.perf_counter()
    return pi_estimate, end - start

def monte_carlo_worker(n_samples_per_process):
    """Worker function para cada processo"""
    np.random.seed()  # Seed diferente para cada processo
    x = np.random.uniform(-1, 1, n_samples_per_process)
    y = np.random.uniform(-1, 1, n_samples_per_process)
    
    inside_circle = (x**2 + y**2) <= 1
    return np.sum(inside_circle)

def monte_carlo_pi_parallel(n_samples, n_processes):
    """Estimativa de π paralela usando multiprocessing"""
    start = time.perf_counter()
    
    samples_per_process = n_samples // n_processes
    
    with ProcessPoolExecutor(max_workers=n_processes) as executor:
        # Submeter tarefas para todos os processos
        futures = [executor.submit(monte_carlo_worker, samples_per_process) 
                  for _ in range(n_processes)]
        
        # Coletar resultados
        total_inside = sum(future.result() for future in as_completed(futures))
    
    pi_estimate = 4 * total_inside / n_samples
    end = time.perf_counter()
    return pi_estimate, end - start

# Teste com diferentes números de amostras
sample_sizes = [1_000_000, 10_000_000, 50_000_000]
n_processes = mp.cpu_count()

print(f"Estimando π usando Monte Carlo com {n_processes} processos")
print("=" * 60)

results = []
for n_samples in sample_sizes:
    print(f"\\nAmostras: {n_samples:,}")
    
    # Serial
    pi_serial, time_serial = monte_carlo_pi_serial(n_samples)
    
    # Paralelo
    pi_parallel, time_parallel = monte_carlo_pi_parallel(n_samples, n_processes)
    
    speedup = time_serial / time_parallel
    efficiency = speedup / n_processes
    error_serial = abs(pi_serial - np.pi)
    error_parallel = abs(pi_parallel - np.pi)
    
    print(f"  Serial:   π ≈ {pi_serial:.6f}, erro = {error_serial:.6f}, tempo = {time_serial:.3f}s")
    print(f"  Paralelo: π ≈ {pi_parallel:.6f}, erro = {error_parallel:.6f}, tempo = {time_parallel:.3f}s")
    print(f"  Speedup: {speedup:.2f}x, Eficiência: {efficiency:.2f}")
    
    results.append({
        'samples': n_samples,
        'speedup': speedup,
        'efficiency': efficiency,
        'time_serial': time_serial,
        'time_parallel': time_parallel
    })

print(f"\\n📊 Valor real de π: {np.pi:.6f}")
print(f"💡 Speedup teórico máximo: {n_processes}x")

In [ ]:
# Visualização dos resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico de Speedup
samples = [r['samples'] for r in results]
speedups = [r['speedup'] for r in results]
efficiencies = [r['efficiency'] for r in results]

ax1.plot(samples, speedups, 'bo-', label='Speedup Real', linewidth=2, markersize=8)
ax1.axhline(y=n_processes, color='r', linestyle='--', label=f'Speedup Ideal ({n_processes}x)')
ax1.set_xlabel('Número de Amostras')
ax1.set_ylabel('Speedup')
ax1.set_title('Speedup vs Tamanho do Problema')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_xscale('log')

# Gráfico de Eficiência
ax2.plot(samples, efficiencies, 'go-', label='Eficiência', linewidth=2, markersize=8)
ax2.axhline(y=1.0, color='r', linestyle='--', label='Eficiência Ideal (1.0)')
ax2.set_xlabel('Número de Amostras')
ax2.set_ylabel('Eficiência')
ax2.set_title('Eficiência vs Tamanho do Problema')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_xscale('log')
ax2.set_ylim(0, 1.1)

plt.tight_layout()
plt.show()

print("\\n🔍 Observações:")
print("• Speedup próximo ao ideal para problemas grandes")
print("• Eficiência diminui com overhead de comunicação")
print("• Monte Carlo é embaraçosamente paralelo (ideal para multiprocessing)")

## 5. Exemplo 3: Multiplicação de Matrizes por Blocos

A multiplicação de matrizes é fundamental em simulações de engenharia (FEM, análise estrutural).

In [ ]:
def matrix_multiply_block_worker(args):
    """Worker que multiplica um bloco da matriz"""
    A_block, B, start_row, end_row = args
    return start_row, np.dot(A_block, B)

def matrix_multiply_parallel(A, B, n_processes):
    """Multiplicação de matrizes paralela por blocos de linhas"""
    start = time.perf_counter()
    
    rows_per_process = A.shape[0] // n_processes
    
    with ProcessPoolExecutor(max_workers=n_processes) as executor:
        # Dividir matriz A em blocos de linhas
        tasks = []
        for i in range(n_processes):
            start_row = i * rows_per_process
            end_row = start_row + rows_per_process if i < n_processes - 1 else A.shape[0]
            
            A_block = A[start_row:end_row, :]
            tasks.append((A_block, B, start_row, end_row))
        
        # Submeter tarefas
        futures = [executor.submit(matrix_multiply_block_worker, task) for task in tasks]
        
        # Coletar resultados
        results = [future.result() for future in as_completed(futures)]
    
    # Reconstituir matriz resultado
    results.sort(key=lambda x: x[0])  # Ordenar por start_row
    C = np.vstack([result[1] for result in results])
    
    end = time.perf_counter()
    return C, end - start

def matrix_multiply_serial(A, B):
    """Multiplicação serial usando NumPy"""
    start = time.perf_counter()
    C = np.dot(A, B)
    end = time.perf_counter()
    return C, end - start

# Teste com matrizes de diferentes tamanhos
matrix_sizes = [512, 1024, 2048]
processes_to_test = [1, 2, 4, mp.cpu_count()]

print("Multiplicação de Matrizes: Serial vs Paralelo")
print("=" * 50)

for size in matrix_sizes:
    print(f"\\n📊 Matrizes {size}x{size}")
    
    # Criar matrizes aleatórias
    np.random.seed(42)
    A = np.random.randn(size, size).astype(np.float32)
    B = np.random.randn(size, size).astype(np.float32)
    
    # Teste serial (NumPy)
    C_serial, time_serial = matrix_multiply_serial(A, B)
    print(f"  NumPy (serial):     {time_serial:.3f}s")
    
    # Teste paralelo com diferentes números de processos
    for n_proc in processes_to_test:
        if n_proc == 1:
            continue  # Já testamos serial
            
        C_parallel, time_parallel = matrix_multiply_parallel(A, B, n_proc)
        speedup = time_serial / time_parallel
        
        # Verificar se os resultados são equivalentes
        are_equal = np.allclose(C_serial, C_parallel, rtol=1e-5)
        status = "✓" if are_equal else "✗"
        
        print(f"  {n_proc} processos:      {time_parallel:.3f}s, speedup: {speedup:.2f}x {status}")

print("\\n⚠️  Note: NumPy já usa BLAS otimizado (pode ser paralelo internamente)")

## 6. Aplicações em Engenharia Civil

### 🏗️ Finite Element Method (FEM) Paralelo

Um dos principais usos de HPC em engenharia civil é na análise estrutural via elementos finitos.

In [ ]:
def simulate_beam_deflection_worker(args):
    """
    Simula deflexão de uma viga para um conjunto de cargas
    Simplificação: viga engastada com carga concentrada
    """
    loads, beam_length, E, I = args
    deflections = []
    
    for P in loads:
        # Deflexão máxima: δ = PL³/(3EI) para viga engastada
        delta_max = (P * beam_length**3) / (3 * E * I)
        deflections.append(delta_max)
    
    return deflections

def parallel_structural_analysis(loads_array, beam_length, E, I, n_processes):
    """
    Análise estrutural paralela para múltiplas configurações de carga
    """
    start = time.perf_counter()
    
    # Dividir cargas entre processos
    chunk_size = len(loads_array) // n_processes
    
    with ProcessPoolExecutor(max_workers=n_processes) as executor:
        tasks = []
        for i in range(n_processes):
            start_idx = i * chunk_size
            end_idx = start_idx + chunk_size if i < n_processes - 1 else len(loads_array)
            
            loads_chunk = loads_array[start_idx:end_idx]
            tasks.append((loads_chunk, beam_length, E, I))
        
        # Executar simulações
        futures = [executor.submit(simulate_beam_deflection_worker, task) for task in tasks]
        results = [future.result() for future in as_completed(futures)]
    
    # Combinar resultados
    all_deflections = []
    for result in results:
        all_deflections.extend(result)
    
    end = time.perf_counter()
    return np.array(all_deflections), end - start

# Exemplo: Análise de uma viga para diferentes cargas
print("🏗️  Simulação: Análise Estrutural Paralela")
print("=" * 45)

# Parâmetros da viga (aço estrutural)
beam_length = 5.0  # metros
E = 200e9  # GPa (módulo de elasticidade do aço)
I = 1e-4   # m⁴ (momento de inércia)

# Gerar diferentes configurações de carga
n_simulations = 100_000
np.random.seed(42)
loads_array = np.random.uniform(1000, 50000, n_simulations)  # N

print(f"Analisando {n_simulations:,} configurações de carga...")
print(f"Viga: L = {beam_length}m, E = {E/1e9:.0f} GPa, I = {I*1e6:.1f} cm⁴")

# Análise serial
start = time.perf_counter()
deflections_serial = []
for P in loads_array:
    delta = (P * beam_length**3) / (3 * E * I)
    deflections_serial.append(delta)
deflections_serial = np.array(deflections_serial)
time_serial = time.perf_counter() - start

# Análise paralela
deflections_parallel, time_parallel = parallel_structural_analysis(
    loads_array, beam_length, E, I, mp.cpu_count()
)

# Verificar resultados
are_equal = np.allclose(deflections_serial, deflections_parallel)
speedup = time_serial / time_parallel

print(f"\\nResultados:")
print(f"  Serial:   {time_serial:.3f}s")
print(f"  Paralelo: {time_parallel:.3f}s")
print(f"  Speedup:  {speedup:.2f}x")
print(f"  Precisão: {'✓' if are_equal else '✗'}")

# Estatísticas dos resultados
print(f"\\nEstatísticas das deflexões:")
print(f"  Mínima:  {np.min(deflections_parallel)*1000:.2f} mm")
print(f"  Máxima:  {np.max(deflections_parallel)*1000:.2f} mm")
print(f"  Média:   {np.mean(deflections_parallel)*1000:.2f} mm")

# Visualização
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(loads_array/1000, bins=50, alpha=0.7, color='blue', edgecolor='black')
plt.xlabel('Carga (kN)')
plt.ylabel('Frequência')
plt.title('Distribuição das Cargas')
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.hist(deflections_parallel*1000, bins=50, alpha=0.7, color='red', edgecolor='black')
plt.xlabel('Deflexão (mm)')
plt.ylabel('Frequência')
plt.title('Distribuição das Deflexões')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\\n💡 Aplicações Reais:")
print("• Montagem paralela de matrizes de rigidez")
print("• Análise modal distribuída")
print("• Simulações Monte Carlo para confiabilidade estrutural")
print("• Otimização de designs estruturais")

## 7. Monitoramento de Performance e Recursos

Vamos monitorar o uso de CPU e memória durante a execução paralela.

In [ ]:
def monitor_system_resources():
    """Monitora uso de CPU e memória"""
    cpu_percent = psutil.cpu_percent(interval=0.1)
    memory = psutil.virtual_memory()
    
    print(f"💻 Recursos do Sistema:")
    print(f"   CPU: {cpu_percent:.1f}% em uso")
    print(f"   RAM: {memory.percent:.1f}% em uso ({memory.used // (1024**3):.1f}GB / {memory.total // (1024**3):.1f}GB)")
    print(f"   Processos disponíveis: {mp.cpu_count()}")

def cpu_intensive_task(duration=2):
    """Tarefa intensiva em CPU para demonstração"""
    start = time.perf_counter()
    count = 0
    while time.perf_counter() - start < duration:
        count += 1
    return count

print("🔍 Monitoramento antes da execução:")
monitor_system_resources()

print("\\n🚀 Executando tarefa CPU-intensiva em paralelo...")

# Executar tarefa intensiva em todos os núcleos
with ProcessPoolExecutor(max_workers=mp.cpu_count()) as executor:
    futures = [executor.submit(cpu_intensive_task, 1) for _ in range(mp.cpu_count())]
    
    # Monitorar durante execução
    print("\\n📊 Durante execução paralela:")
    time.sleep(0.5)  # Aguardar processos começarem
    monitor_system_resources()
    
    # Aguardar conclusão
    results = [future.result() for future in as_completed(futures)]

print("\\n✅ Após execução:")
monitor_system_resources()

print(f"\\n📈 Operações realizadas por processo: {results}")
print(f"    Total de operações: {sum(results):,}")

## 8. Resumo e Próximos Passos

### ✅ O que aprendemos hoje:

1. **Conceitos fundamentais de paralelismo**
   - Speedup, eficiência, overhead
   - Lei de Amdahl e Lei de Gustafson

2. **Limitações do Python para paralelismo**
   - Global Interpreter Lock (GIL)
   - Threads vs Processos

3. **Multiprocessing em Python**
   - `ProcessPoolExecutor` para paralelização
   - Comunicação entre processos

4. **Aplicações práticas**
   - Monte Carlo paralelo
   - Multiplicação de matrizes por blocos
   - Simulações estruturais

5. **Medição de performance**
   - Timing adequado com `time.perf_counter()`
   - Monitoramento de recursos do sistema

### 🚀 Próxima aula:
- Ferramentas avançadas: `joblib`, `numba`
- Análise de escalabilidade
- Vetorização e SIMD
- Limites práticos do paralelismo

### 📝 Tarefa para casa:
Paralelizar um código próprio ou reproduzir os exemplos com dados diferentes.